In [1]:
import os
from time import time
from pathlib import Path
import pandas as pd 
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from datasets import Dataset
import accelerate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score

Obtenemos la carpeta actual:

In [2]:
current_dir = Path.cwd()
DATASETS_LOCATION = os.path.join(current_dir.parent.parent.parent, 'datasets')
MODELS_LOCATION = os.path.join(current_dir.parent.parent, 'models')

Cargamos el dataset en un dataframe:

In [3]:
df_disease_sp = pd.read_excel(os.path.join(DATASETS_LOCATION, 'disease_nlp_eng.xlsx'))
df_disease_sp.head(3)

,Symptoms,Disease
0,I've been feeling tired all the time and notic...,Anemia
1,"Lately, I’ve been out of breath even after lig...",Anemia
2,"For the past few weeks, I’ve been extremely we...",Anemia


In [4]:
print(df_disease_sp['Disease'].value_counts())

Disease
Anemia         100
Thalassemia    100
Thrombosis     100
Diabetes       100
Name: count, dtype: int64


Observamos que ya las clases están bien distribuidas. Dividimos el dataset en conjunto de entrenamiento y prueba, y a su vez el conjunto de entrenamiento lo dividiremos en conjunto de entrenamiento y conjunto de validación:

In [5]:
train_df, test_df = train_test_split(df_disease_sp, test_size=0.2, random_state=42, stratify=df_disease_sp['Disease'])
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['Disease'])

print("\nConjunto de entrenamiento:")
print(train_df['Disease'].value_counts())

print("\nConjunto de validación:")
print(val_df['Disease'].value_counts())

print("\nConjunto de prueba:")
print(test_df['Disease'].value_counts())


Conjunto de entrenamiento:
Disease
Thrombosis     64
Anemia         64
Thalassemia    64
Diabetes       64
Name: count, dtype: int64

Conjunto de validación:
Disease
Thrombosis     16
Thalassemia    16
Anemia         16
Diabetes       16
Name: count, dtype: int64

Conjunto de prueba:
Disease
Thalassemia    20
Diabetes       20
Anemia         20
Thrombosis     20
Name: count, dtype: int64


Codificamos las clases de 'Disease', utilizando LabelEncoder:

In [6]:
label_encoder = LabelEncoder()

train_df['label'] = label_encoder.fit_transform(train_df['Disease'])
val_df['label'] = label_encoder.transform(val_df['Disease'])
test_df['label'] = label_encoder.transform(test_df['Disease'])

train_df.head()

,Symptoms,Disease,label
281,"I’ve noticed my leg is swollen, and the veins ...",Thrombosis,3
22,"I’ve been getting infections a lot, and my ski...",Anemia,0
146,"I’ve been so weak lately, and my skin looks ye...",Thalassemia,2
70,I’ve noticed that the whites of my eyes have a...,Anemia,0
376,"My vision’s been blurry, I’m constantly thirst...",Diabetes,1


## Entrenamiento

Definamos la función de métricas:

In [7]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "precision": precision_score(p.label_ids, preds, average='macro'),
        "recall": recall_score(p.label_ids, preds, average='macro'),
        "f1": f1_score(p.label_ids, preds, average='macro'),
    }

Definimos el modelo y el tokenizador. En este caso, utilizaremos el modelo preentrenado de hugging face 'Distilbert':

In [8]:
model_checkpoint = "distilbert-base-uncased"  
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=4)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Preprocesamos los datos
def preprocess_function(examples):
    return tokenizer(examples['Symptoms'], padding=True, truncation=True, return_tensors="pt")

Convertimos los dataframe en datasets de hugging face, y los tokenizamos:

In [10]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
validation_dataset = Dataset.from_pandas(val_df)

train_preprocessed_dataset = train_dataset.map(preprocess_function, batched=True)
test_preprocessed_dataset = test_dataset.map(preprocess_function, batched=True)
val_preprocessed_dataset = validation_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/256 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

In [11]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,              # Tasa de aprendizaje: se ha probado 5e-6, 1e-5, 5e-5, 2e-5
    per_device_train_batch_size=16,   # Tamaño del batch para entrenamiento: se ha probado 8, 16, 32, 64
    per_device_eval_batch_size=16,    # Tamaño del batch para evaluación
    num_train_epochs=7,
    weight_decay=0.2,               # Decaimiento de peso: hemos probado 0.01, 0.03, 0.1, 0.2 e 0.3 e 0.4 y este era el mejor
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_preprocessed_dataset,
    eval_dataset=val_preprocessed_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics   # Función para calcular las métricas
)

c:\Users\maria\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\maria\AppData\Local\Temp\ipykernel_764\66794223.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Training

Y entrenamos el modelo:

In [12]:
start = time()

trainer.train()

end = time()
print(f">>>>>>>>>>>>> elapsed time: {(end-start)/60:.0f}m")

  0%|          | 0/112 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 1.2753331661224365, 'eval_accuracy': 0.515625, 'eval_precision': 0.6634199134199135, 'eval_recall': 0.515625, 'eval_f1': 0.4590954612203386, 'eval_runtime': 0.8381, 'eval_samples_per_second': 76.365, 'eval_steps_per_second': 4.773, 'epoch': 1.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.9782110452651978, 'eval_accuracy': 0.625, 'eval_precision': 0.7351532567049808, 'eval_recall': 0.625, 'eval_f1': 0.5782099936748893, 'eval_runtime': 0.8306, 'eval_samples_per_second': 77.049, 'eval_steps_per_second': 4.816, 'epoch': 2.0}


...

{'eval_loss': 0.4030015468597412, 'eval_accuracy': 0.953125, 'eval_precision': 0.9605263157894737, 'eval_recall': 0.953125, 'eval_f1': 0.9527093596059113, 'eval_runtime': 0.8347, 'eval_samples_per_second': 76.678, 'eval_steps_per_second': 4.792, 'epoch': 5.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.3617309331893921, 'eval_accuracy': 0.953125, 'eval_precision': 0.9555555555555555, 'eval_recall': 0.953125, 'eval_f1': 0.9530360531309299, 'eval_runtime': 0.891, 'eval_samples_per_second': 71.826, 'eval_steps_per_second': 4.489, 'epoch': 6.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.33066028356552124, 'eval_accuracy': 0.96875, 'eval_precision': 0.9722222222222222, 'eval_recall': 0.96875, 'eval_f1': 0.9686274509803922, 'eval_runtime': 1.211, 'eval_samples_per_second': 52.848, 'eval_steps_per_second': 3.303, 'epoch': 7.0}
{'train_runtime': 117.1139, 'train_samples_per_second': 15.301, 'train_steps_per_second': 0.956, 'train_loss': 0.7024098805018834, 'epoch': 7.0}
>>>>>>>>>>>>> elapsed time: 2m


## Evaluación

Evaluamos el modelo en el conjunto de prueba:

In [14]:
# Preprocesamos el conjunto de prueba
test_preprocessed_dataset = test_dataset.map(preprocess_function, batched=True)

# Evaluamos el modelo en el conjunto de prueba
results = trainer.evaluate(test_preprocessed_dataset)

# Imprimimos las métricas de evaluación
print("Resultados de la evaluación en el conjunto de prueba:")
print(f"Exactitud: {results['eval_accuracy']:.4f}")
print(f"Precisión: {results['eval_precision']:.4f}")
print(f"Recall: {results['eval_recall']:.4f}")
print(f"F1-Score: {results['eval_f1']:.4f}")

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Resultados de la evaluación en el conjunto de prueba:
Exactitud: 0.9625
Precisión: 0.9641
Recall: 0.9625
F1-Score: 0.9625


# Guardar el modelo

Guardamos el modelo y el tokenizer:

In [15]:
save_directory = os.path.join(MODELS_LOCATION, 'disease_classification_english_nlp')
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
print(f"Modelo y tokenizer guardados en el directorio {save_directory}")

Modelo y tokenizer guardados en el directorio c:\Users\maria\Desktop\universidad\master\TFM\tfm\src\models\disease_classification_english_nlp


Definamos algunas frases de ejemplo para el test, que puedan confundir a nuestro modelo:

In [16]:
sample_sentences = [
    {"Symptoms": "My vision has been blurry lately, and I feel so drained.", "Disease": "Diabetes"},
    {"Symptoms": "I’ve been feeling unusually weak and cold all the time.", "Disease": "Anemia"},
    {"Symptoms": "There’s a strange heaviness in my leg, and it’s very swollen.", "Disease": "Thrombosis"},
    {"Symptoms": "My skin looks yellowish, and I often feel short of breath.", "Disease": "Thalassemia"},
    {"Symptoms": "I get out of breath even when doing small tasks, and my heart feels like it’s racing.", "Disease": "Anemia"},
    {"Symptoms": "I’ve noticed a lot of pain in my calf, and it seems warmer than usual.", "Disease": "Thrombosis"}
]

# Conveirtamos las frases de ejemplo en un dataframe
sample_df = pd.DataFrame(sample_sentences)

# Tokenizamos las frases de ejemplo
sample_dataset = Dataset.from_pandas(sample_df)
sample_preprocessed_dataset = sample_dataset.map(preprocess_function, batched=True)

# Efectuemos las predicciones
sample_predictions = trainer.predict(sample_preprocessed_dataset)
sample_preds = np.argmax(sample_predictions.predictions, axis=1)
sample_df["Predicted disease"] = label_encoder.inverse_transform(sample_preds)

print("Resultados de las predicciones en las frases de ejemplo:")
sample_df

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Resultados de las predicciones en las frases de ejemplo:


,Symptoms,Disease,Predicted disease
0,"My vision has been blurry lately, and I feel s...",Diabetes,Diabetes
1,I’ve been feeling unusually weak and cold all ...,Anemia,Anemia
2,"There’s a strange heaviness in my leg, and it’...",Thrombosis,Thrombosis
3,"My skin looks yellowish, and I often feel shor...",Thalassemia,Diabetes
4,I get out of breath even when doing small task...,Anemia,Diabetes
5,"I’ve noticed a lot of pain in my calf, and it ...",Thrombosis,Thrombosis


# Test de Carga

In [17]:
model_directory = os.path.join(MODELS_LOCATION, 'disease_classification_english_nlp')
# Carica il tokenizer e il modello
tokenizer = AutoTokenizer.from_pretrained(model_directory, local_files_only=True)
model_loaded = AutoModelForSequenceClassification.from_pretrained(model_directory, local_files_only=True)
def classify_disease_symptoms(input_text: str) -> str:
    prediction_mapping = {0: 'Anemia',
                          1: 'Diabetes', 
                          2: 'Thalassemia', 
                          3: 'Thrombosis'}
    # Tokenizza la frase
    inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt")
    # Effettua la predizione
    with torch.no_grad():
        outputs = model_loaded(**inputs)
        prediction = torch.argmax(outputs.logits, axis=1).item()
    return prediction_mapping[prediction]


# Aggiungi una colonna con le predizioni
def add_predictions(df):
    df['Predicted Disease'] = df['Symptoms'].apply(classify_disease_symptoms)
    return df

# Aggiorna il dataframe con le predizioni
predizioni = add_predictions(test_df)
# Estrai i valori reali e predetti
true_values = predizioni['Disease']
predicted_values = predizioni['Predicted Disease']

# Calcola l'accuracy
accuracy = accuracy_score(true_values, predicted_values)

# Mostra il risultato
print(f"L'accuracy del modello è: {accuracy:.2f}")


L'accuracy del modello è: 0.96
